<a href="https://colab.research.google.com/github/singclare/PracticePython/blob/master/%ED%85%94%EB%A0%88%EA%B7%B8%EB%9E%A8_%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80_%EC%95%8C%EB%A6%BC%EB%B4%87_%EC%A0%9C%EC%9E%91_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install schedule

In [ ]:
# HTTP 통신
import requests
from urllib.parse import urlencode, quote_plus, urlparse
from urllib.request import urlopen
import urllib

# 데이터 가공
from bs4 import BeautifulSoup
import json

# 시간 관련 
import datetime
from time import sleep
import schedule

## 1. 텔레그램 연동, API 활용
![대체 텍스트](https://postfiles.pstatic.net/MjAxOTEwMjJfODkg/MDAxNTcxNzI0NzU5NzQ3.DWq-KV-5JUKFWAkcWuRLKeDFFXSgcolN7uXsMepnkqAg.sbTVzFhJGTiAhm8b34omsXqWwzn45knyzPPubT_lKbIg.PNG.rlatlawkd/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2019-10-22_%EC%98%A4%ED%9B%84_3.11.36.png?type=w773)

1. 텔레그램 설치
2. BotFather 채널 추가
3. 나의 봇 생성
4. API key 발급

참고
https://kimdondon.tistory.com/17

In [ ]:
### 각자 할당 받은 API 키로 수정하자. ###

API_key_Telegram = '1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg'

In [ ]:
def get_myChatID() :
    
    # 이 코드셀에서 에러가 난다면, 해당 텔레그램 봇에서 직접 아무 메시지나 "두 번" 입력

    endpoint = 'https://api.telegram.org/bot'
    query = '/' + 'getUpdates'
    URL = endpoint + API_key_Telegram + query
    
    print(URL)
    
    request = urllib.request.Request(URL)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    request_body = urlopen(request).read()

    if rescode == 200 :
        print(rescode)
        print('SUCCESS \n')
        chat_id = json.loads(request_body)['result'][0]['message']['from']['id']
        print('나의 chat ID : '+ str(chat_id))
     
        return str(chat_id)
    
    else :
        print(rescode)
        print('FAILED')

In [ ]:
# 이 함수가 에러 없이 실행되면 텔레그램에 무사히 연동된 것이다.

get_myChatID()

https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/getUpdates
200
SUCCESS 

나의 chat ID : 825106853


'825106853'

### 에러가 여러가지 형태로 날 수 있다.

HTTPError: HTTP Error 401: Unauthorized
- API Key가 올바른지 확인해 보자.

IndexError: list index out of range
- 텔레그램에서 아무 채팅이나 2번 이상 입력

HTTPError: HTTP Error 502: Bad Gateway
- 엔드 포인트 에러. 강사가 직접 봐야 함.

In [ ]:
def send_message(input_text) :
    
    # 유저마다 chat id가 다르다.
    chat_id = get_myChatID()
    
    endpoint = 'https://api.telegram.org/bot'
    action = '/' + 'sendmessage'
    param_list = [
        'chat_id=' + str(chat_id), 
        'text=' + input_text
    ]
    param = '&'.join(param_list)
    query = action + '?' + param
    
    URL = endpoint + API_key_Telegram + query
    print(URL)
    
    try : 
        r = requests.get(URL)
        print(r.text)
    
    except Exception as e:
        print(str(e))

In [ ]:
send_message('TEST TEST TEST 123 123 안녕하세요')

https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/getUpdates
200
SUCCESS 

나의 chat ID : 825106853
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/sendmessage?chat_id=825106853&text=TEST TEST TEST 123 123 안녕하세요
{"ok":true,"result":{"message_id":7,"from":{"id":1067658259,"is_bot":true,"first_name":"simin_misemise_bot","username":"simin_misemise_bot"},"chat":{"id":825106853,"first_name":"\uc2dc\ubbfc","last_name":"\uae40","type":"private"},"date":1580455512,"text":"TEST TEST TEST 123 123 \uc548\ub155\ud558\uc138\uc694"}}


## 2. 공공데이터 API 활용

![](https://postfiles.pstatic.net/MjAxOTEwMjJfMjQ4/MDAxNTcxNzI0NzU5NzQ4.9OcNdlW37QlWHm5-8AA8wZLw8Mj9OsP3vwsbINu399Ug.LrpiCfa6KKbCk4KaWd8Z9WtgTMi2d_F_NECogGzXxtUg.PNG.rlatlawkd/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2019-10-22_%EC%98%A4%ED%9B%84_3.09.25.png?type=w773)

In [ ]:
### 각자 할당 받은 API 키로 수정하자. ###

#API_key_opendata = '4uLYNT4VBfdIchhsd0UX57z9i6WMo4k2KjD126qoi00ZCbyheNCSBCV4bFwZDcV75DizH2nYTtvWyk400xS56A%3D%3D'
API_key_opendata = 'pKdeOUo9aa7hDWmQ4RfZ%2B84zx7eAQjsOuhtAD94DLH9nAoiDxc3CFLAtdBarqYAkxIBgwd7XxIgvHB8lXtGgEg%3D%3D'

In [ ]:
def getData_airPollution() :
    
    # API : 대기오염정보 조회 서비스
    endpoint = 'http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureSidoLIst'
    
    headers_params = { 
    #quote_plus('ServiceKey') : api_key_utf8, 
    # Service Key는 특수문자를 포함하고 있기 때문에 인코딩 parameter 객체에 넣기에는 인코딩 과정이 복잡하다.
    # 아래 코드 셀에서 직접 URL의 Service Key 부분만 수동으로 타이핑하자.
        quote_plus('ServiceKey') : API_key_opendata, 
        quote_plus('pageNo') : '1', 
        quote_plus('numOfRows') : '50', 
        quote_plus('sidoName') : '대전', ### 도시명 수정 가능 ###
        quote_plus('searchCondition') : 'HOUR'
    }

    queryParams = ('?' + 
                  'ServiceKey=' + API_key_opendata + '&' + 
                  urllib.parse.urlencode(headers_params, 'UTF-8'))

    url = endpoint  + queryParams
    
    request = urllib.request.Request(url)
    request.get_method = lambda : 'GET'

    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    request_body = urlopen(request).read()
    print('request 완료\n')
    
    # 여기까지 API 호출 코드였다. #
    # 아래부터 API로 받아온 데이터를 가공하는 작업 --> 시간, 지역구, 각종 대기오염 정보 #
    
    request_soup = BeautifulSoup(request_body, 'html.parser')
    
    ### 지역구 변경 가능
    index_gu = 2

    ### <서울>
    ### 0 강남구, 1 강동구, 2 강북구, 3 강서구, 4 관악구,
    ### 5 광진구, 6 구로구, 7 금천구, 8 노원구, 9 도봉구,
    ### 10 동대문구, 11 동작구, 12 마포구, 13 서대문구, 14 서초구,
    ### 15 성동구, 16 성북구, 17 송파구, 18 양천구, 19 영등포구,
    ### 20 용산구, 21 은평구, 22 종로구, 23 중구, 24 중랑구

    ### <대전>
    ### 0 대덕구, 1 동구, 2 서구, 3 유성구, 4 중구
    
    ### <대구>
    ### 0 남구, 1 달서구, 2 달성군, 3 동구, 4 북구, 
    ### 5 서구, 6 수성구, 7 중구

    result_dict = {
        'datatime' : request_soup.body.find_all('datatime')[index_gu].text, # 데이터 최근 업데이트 시간
        'cityname' : request_soup.body.find_all('cityname')[index_gu].text, # 지역구
        'so2value' : request_soup.body.find_all('so2value')[index_gu].text, # 아황산가스 평균농도,
        'covalue' : request_soup.body.find_all('covalue')[index_gu].text, # 일산화탄소 평균농도 
        'o3value' : request_soup.body.find_all('o3value')[index_gu].text, # 오존 평균농도
        'no2value' : request_soup.body.find_all('no2value')[index_gu].text, # 이산화질소 평균농도
        'pm10value' : request_soup.body.find_all('pm10value')[index_gu].text, # 미세먼지 PM10 평균농도
        'pm25value' : request_soup.body.find_all('pm25value')[index_gu].text # 미세먼지 PM2.5 평균농도
    }
        
      
    
    return result_dict
    print(result_dict)

In [ ]:
getData_airPollution()

request 완료



{'cityname': '서구',
 'covalue': '0.3',
 'datatime': '2020-08-13 22:00',
 'no2value': '0.015',
 'o3value': '0.026',
 'pm10value': '33',
 'pm25value': '14',
 'so2value': '0.002'}

## 3. 공공데이터 가공 -> 기사문

![](https://postfiles.pstatic.net/MjAxOTEwMjJfMjU5/MDAxNTcxNzI0NzU5NzQ3.ncPtuGErFTQ1WNqYGtkczA0GncphiNsEJyND2Btho3sg.IfiAjfOlBD42o2fuzNE2vYRay3sFL2B1segbAA6c6Sog.PNG.rlatlawkd/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2019-10-22_%EC%98%A4%ED%9B%84_3.09.51.png?type=w773)

In [ ]:
def process_to_article() : 

    dict_to_process = getData_airPollution()
    # 위에서 이미 정의한, 데이터 수집 함수(getData_airPollution)가 포함된다.
    # 즉, 데이터 기사문으로 가공하는 이 함수를 실행하면, 자동으로 데이터 수집을 먼저 진행하게 된다.

    
    ### 아래 코드는 눈에 보이는 메시지로, 구체적인 형태로 가공하는 작업
    text_result =  (
        dict_to_process['datatime'] + ' 기준' + '\n' + 
        dict_to_process['cityname'] + '의 미세먼지 농도는' + '\n' + 
        'PM10 : ' + dict_to_process['pm10value'] + 'ppm' + '\n' + 
        'PM2.5 : ' + dict_to_process['pm25value'] + 'ppm'
    )
    
    print(text_result)
    return text_result
   

In [ ]:
process_to_article()

request 완료

2020-01-31 16:00 기준
서구의 미세먼지 농도는
PM10 : 41ppm
PM2.5 : 25ppm


'2020-01-31 16:00 기준\n서구의 미세먼지 농도는\nPM10 : 41ppm\nPM2.5 : 25ppm'

## 4. 공공데이터 기사문 -> 텔레그램 전송

![](https://postfiles.pstatic.net/MjAxOTEwMjJfMTkz/MDAxNTcxNzI2NjU1NDcx.x3vPqfGHtnY9NW86lHvuY8GIQUwCC1HL7gFTLvLL7Cgg.KEnDUuGG2OWoxthJZK_Lbd9Uik2V8GNuJglKN7RjsQ8g.PNG.rlatlawkd/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2019-10-22_%EC%98%A4%ED%9B%84_3.43.57.png?type=w773)

In [ ]:
def message_article():
  
  article = process_to_article()
  # 위에서 이미 정의한, 데이터 수집 함수(process_to_article)가 포함된다.
  # 즉, 기사문을 텔레그램으로 전송하는 이 함수를 실행하면, 
  # 자동으로 데이터 수집을 먼저, 그 다음으로는 수집한 데이터를 기사문으로 가공하는 작업을 진행하게 된다.

  send_message(article)

In [ ]:
# 최종 완결 형태인 함수 하나가 만들어졌다.
# 아래 함수를 실행하면 우리 텔레그램으로 기사문이 전송된다. 

message_article()

request 완료

2020-01-31 16:00 기준
서구의 미세먼지 농도는
PM10 : 41ppm
PM2.5 : 25ppm
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/getUpdates
200
SUCCESS 

나의 chat ID : 825106853
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/sendmessage?chat_id=825106853&text=2020-01-31 16:00 기준
서구의 미세먼지 농도는
PM10 : 41ppm
PM2.5 : 25ppm
{"ok":true,"result":{"message_id":9,"from":{"id":1067658259,"is_bot":true,"first_name":"simin_misemise_bot","username":"simin_misemise_bot"},"chat":{"id":825106853,"first_name":"\uc2dc\ubbfc","last_name":"\uae40","type":"private"},"date":1580457043,"text":"2020-01-31 16:00 \uae30\uc900\n\uc11c\uad6c\uc758 \ubbf8\uc138\uba3c\uc9c0 \ub18d\ub3c4\ub294\nPM10 : 41ppm\nPM2.5 : 25ppm"}}


![](https://postfiles.pstatic.net/MjAxOTEwMjJfMTM0/MDAxNTcxNzI0NzU5NzM5.DsMvbeGbP8twe3zGyl8ax5EDqxBx5JFkrW-RHdLY3-gg.S4WF5N0n7fxcqSeBJf7Pmr3J_jBwJkmBCYskYQ-MTaAg.PNG.rlatlawkd/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2019-10-22_%EC%98%A4%ED%9B%84_3.12.15.png?type=w773)

이제는 매 1분(=60초) 마다 같은 함수가 돌아가도록 해보자. 

In [ ]:
while True : # Always
  message_article()
  sleep(10)

request 완료

2020-01-31 16:00 기준
서구의 미세먼지 농도는
PM10 : 41ppm
PM2.5 : 25ppm
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/getUpdates
200
SUCCESS 

나의 chat ID : 825106853
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/sendmessage?chat_id=825106853&text=2020-01-31 16:00 기준
서구의 미세먼지 농도는
PM10 : 41ppm
PM2.5 : 25ppm
{"ok":true,"result":{"message_id":11,"from":{"id":1067658259,"is_bot":true,"first_name":"simin_misemise_bot","username":"simin_misemise_bot"},"chat":{"id":825106853,"first_name":"\uc2dc\ubbfc","last_name":"\uae40","type":"private"},"date":1580457139,"text":"2020-01-31 16:00 \uae30\uc900\n\uc11c\uad6c\uc758 \ubbf8\uc138\uba3c\uc9c0 \ub18d\ub3c4\ub294\nPM10 : 41ppm\nPM2.5 : 25ppm"}}
request 완료

2020-01-31 16:00 기준
서구의 미세먼지 농도는
PM10 : 41ppm
PM2.5 : 25ppm
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5PA2TmszfEyfVnrjYXtg/getUpdates
200
SUCCESS 

나의 chat ID : 825106853
https://api.telegram.org/bot1067658259:AAFVrKUfRRNpAJZ5P

KeyboardInterrupt: ignored

 .


### 8월 14일 강의 만족도 조사

 https://forms.gle/5yuQgXCApTkz2eBc9